In [352]:
import pandas as pd
from datetime import datetime
import numpy as np
from datetime import timedelta, date

import matplotlib.dates as mdates
from scipy import stats
import datetime as dt


In [353]:
import psycopg2
connection = psycopg2.connect(user = "postgres", password = "admin", host = "127.0.0.1",port = "5432", database = "postgres")

In [354]:
def process_data(quotes):
    from talib import RSI
    from talib import MACD
    from talib import SMA
    from talib import EMA
    from talib import ATR
    # Simple Moving Averages
    quotes['sma50'] = SMA(quotes['close'], timeperiod=50)
    # Exponential Moving Averages
    quotes['ema50'] = EMA(quotes['close'], timeperiod=50)
    # MACD indicator - MACD(12,26,9)
    quotes['macd_line'], quotes['signal_line'], quotes['macd_hist'] = MACD(quotes['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    # Compute RSI (14 days)
    quotes["rsi"] = RSI(quotes['close'].values, timeperiod=14)
    # Compute ATR (14 days)
    quotes["atr"] = ATR(quotes['high'].values,quotes['low'].values, quotes['close'].values ,timeperiod=14)

    atr_stoploss  = ATR_STOPLOSS(close = quotes.close.values, high = quotes.high.values, low = quotes.low.values, atr = quotes.atr.values, times=5, period=14)
    quotes["atr_stoploss"] = atr_stoploss[0]
    
    return quotes



def ATR_STOPLOSS(close, high, low, atr, times=4, period=14):
    first = 1
    import math
    atr_trailing_stoploss = np.full(len(close)+1, np.nan)
    for i in range(0,len(close)):
        
        if(i>0 and math.isnan(atr[i])==False):
            if(close[i-1]>atr_trailing_stoploss[i] and close[i]<atr_trailing_stoploss[i]):
                loss = atr[i]*times
                atr_trailing_stoploss[i+1]=close[i]+loss  
                continue
        if(i>0 and math.isnan(atr[i])==False):
            if(close[i-1]<atr_trailing_stoploss[i] and close[i]>atr_trailing_stoploss[i]):
                loss = atr[i]*times
                atr_trailing_stoploss[i+1]=close[i]-loss  
                continue
                
        if(math.isnan(atr[i])==False):
            loss = atr[i]*times
            atr_trailing_stoploss[i+1]=close[i]-loss
            if(close[i]<atr_trailing_stoploss[i]):
                if(atr_trailing_stoploss[i]>close[i]+loss):
                    atr_trailing_stoploss[i+1]=close[i]+loss
                else:
                    if(math.isnan(atr_trailing_stoploss[i])==False):
                        atr_trailing_stoploss[i+1]=atr_trailing_stoploss[i]
            else:
                if(atr_trailing_stoploss[i]<close[i]-loss):
                    atr_trailing_stoploss[i+1]=close[i]-loss
                else:
                    if(math.isnan(atr_trailing_stoploss[i])==False):
                        atr_trailing_stoploss[i+1]=atr_trailing_stoploss[i]
    return atr_trailing_stoploss[:-1], atr_trailing_stoploss[-1:]

In [355]:
def close_conn():
    if(connection):
        cursor.close()
        connection.close()

def get_data(stock):
    data = pd.read_sql_query("SELECT * FROM finance.stock_price where stock = '"+stock+"' order by date asc",con=connection)
    return process_data(data)

# 1. Action Signal


In [397]:
def get_action(data, i):
    import random
    action ="HOLD"
    rand = random.uniform(0, 3)
    if(rand < 1):
        action = "LONG"
    if(rand > 2):
        action = "SHORT"
        
        
    if(data.atr_stoploss[i]>data.close[i]):
        action = "SHORT"
    if(data.atr_stoploss[i]<data.close[i]):
        action = "LONG"
    return action

# 2. Back Testing

In [ ]:
def println(str):
    show_each_tries= True
    if(show_each_tries):
        print(str)

def back_test(data, cash1 = 100000,transaction_fee = 0.01, assets={}):
    cash = cash1
    init_cash = cash
    for i,row in data.iterrows():
        #Decision
        action = get_action(data, i)
        #Execution
        if(action=="LONG"):
            shares = int(cash/(row.close*(1+transaction_fee)))
            if(shares>0):
                cost = shares*row.close*(1+transaction_fee)
                cash_left = cash-cost
                if row.stock not in assets:
                    assets[row.stock]=0
                assets[row.stock] = assets[row.stock] + shares
                cash = cash_left
        if(action=="SHORT"):
                if row.stock in assets:
                    shares = assets[row.stock]
                    if(shares>0):
                        asset = shares * row.close
                        cash = cash + asset*(1-transaction_fee)
                        assets.clear()
    final_cash = cash
    if(data.iloc[-1].stock in assets):
        final_cash = assets[data.iloc[-1].stock]*data.iloc[-1].close
    println("asset: "+str(final_cash))
    growth =((final_cash-init_cash)/init_cash)*100
    println("profit: "+str(round(growth,2))+"%")
    return round(growth,2)

total_tries = 10
accumulated_growth = 0
stocks = ["D05.SI","K71U.SI"]
for i in range(0,total_tries):
    for stock in stocks:
        result = back_test(get_data(stock), transaction_fee =0.01)
    accumulated_growth = round(accumulated_growth +result ,2)
print("Average Growth ("+str(total_tries)+"*"+str(len(stocks))+"): "+ str(accumulated_growth/(total_tries+len(stocks)))+"%")

asset: 79487.32
profit: -20.51%
asset: 197411.51514979993
profit: 97.41%
asset: 75414.46
profit: -24.59%
asset: 190635.01352299997
profit: 90.64%
asset: 73817.26000000001
profit: -26.18%
asset: 211352.53965139994
profit: 111.35%
asset: 84278.92
profit: -15.72%
asset: 204573.74688249998
profit: 104.57%
asset: 77703.78
profit: -22.3%
asset: 192893.17522349991
profit: 92.89%
asset: 81856.5
profit: -18.14%
asset: 204027.4119254999
profit: 104.03%
asset: 86009.22
profit: -13.99%
asset: 190636.04978200002
profit: 90.64%
